# Final Model Prediction

In [2]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
import pandas as pd
import json
import numpy as np
import random
from keras.models import load_model
import os
import tensorflow as tf

## Garbage collection

In [78]:
import gc
gc.collect()    

0

## Local Functions first

In [46]:
Normalization=None
def normalize(dataset,file='NoVol_Normalization_Values.json'):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        print('normalization is loaded')

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

## loding the model

In [8]:
model = load_model('XcryptoAi_NoVol.hdf5')

Loding Data

In [7]:
df = pd.read_csv('D:/+DATA+/AICryptoBot/allok_w15_nosell.csv',index_col=0)
high_weight=3
df["high"]=(df["open"]+high_weight*df["high"]+df["low"]+df["close"])/(3+high_weight)
df.rename(columns={"high":"price"},inplace = True)
df["BTC_high"]=(df["BTC_open"]+high_weight*df["BTC_high"]+df["BTC_low"]+df["BTC_close"])/(3+high_weight)
df.rename(columns={"BTC_high":"BTC_price"},inplace = True)
df=df.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])


In [9]:
df.iloc[0]

price          4.87425
volume       241.35000
high-1         4.87480
low-1          4.87420
open-1         4.87420
               ...    
day            5.00000
hour          20.00000
minute        15.00000
lunch_day   -685.00000
buy            0.00000
Name: 0, Length: 759, dtype: float64

Persentage Nomalization

In [10]:
for key in df.keys():
    if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["BTC_price"]-df[key])/df["BTC_price"]
    if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["price"]-df[key])/df["price"]

In [29]:
df['volume'][0]
VolRemover=["volume","volume-1","BTC_volume-1"]
#VolRemover=[]
for key in df.keys():
    if key.find("volume-1_") != -1 :
        VolRemover.append(key)
print(VolRemover)
df=df.drop(columns=VolRemover)

['volume', 'volume-1', 'BTC_volume-1']


Import Normalization values

In [30]:
for k in df.keys():print(k,end='        ')

price        high-1        low-1        open-1        close-1        high-2        low-2        open-2        close-2        volume-2        high-3        low-3        open-3        close-3        volume-3        high-4        low-4        open-4        close-4        volume-4        high-5        low-5        open-5        close-5        volume-5        high-6        low-6        open-6        close-6        volume-6        high-7        low-7        open-7        close-7        volume-7        high-8        low-8        open-8        close-8        volume-8        high-9        low-9        open-9        close-9        volume-9        high-10        low-10        open-10        close-10        volume-10        high-11        low-11        open-11        close-11        volume-11        high-12        low-12        open-12        close-12        volume-12        high-13        low-13        open-13        close-13        volume-13        high-14        low-14        open-14        clo

In [35]:
dt=df.drop(columns=["buy"]).to_numpy()

In [33]:
Y=df["buy"].to_numpy()

In [37]:
print(dt.shape)

(1084382, 747)


In [38]:
print(Y.shape)

(1084382,)


## Try once

In [111]:

index_sample=51541
print(df.iloc[index_sample])
Sample=df.drop(columns=["buy"]).iloc[index_sample]


price          1.455
high-1         0.000
low-1          0.000
open-1         0.000
close-1        0.000
              ...   
day            4.000
hour          18.000
minute        10.000
lunch_day   -709.000
buy            0.000
Name: 51541, Length: 748, dtype: float64


In [112]:
with tf.device('/gpu:0'):
    Bgpu=Buy_Dessision(Sample)

normalization is loaded


In [97]:
with tf.device('/cpu:0'):
    Bcpu=Buy_Dessision(Sample)
Bcpu

normalization is loaded


1

In [113]:
verdict=Bgpu==Y[index_sample]
print("verdict ---> "+str(verdict))

verdict ---> True


## no GPU

In [142]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [71]:
if(df.iloc[33,0]==dt[33,0]):dt=normalize(dt)

normalization is loaded


In [ ]:
with tf.device('/cpu:0'):
    accuracy = model.evaluate(dt, Y)
print("accuracy: "+str(accuracy[1]))


In [105]:
Y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [114]:
with tf.device('/gpu:0'):
    accuracy = model.evaluate(dt[110:int(len(dt)/9),:], Y[110:int(len(dt)/9)])
print("accuracy: "+str(accuracy[1]))


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [115]:
import ccxt
Exchange=ccxt.binance()
ex=Exchange

In [208]:
pair='GMT/BUSD'
bitcoin_ticker = Exchange.fetch_ticker('BTC/USDT')
btc_current_price=bitcoin_ticker['info']['askPrice']
print(btc_current_price)

ex.fetchTime()

18927.24000000


1663793980898

In [191]:
pair_data=pd.DataFrame()

In [199]:
def instant_pair_data(pair="GMT/BUSD",window=15):

    ticker = Exchange.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=15)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=15)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=15)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=15)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=15)


    pair_data.loc[0,"price"]=pair_current_price
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data

def instant_full_data(pair,window=15):
    pdata=instant_pair_data(pair,window=15)
    btcdata=instant_pair_data("BTC/USDT",window=15).add_prefix("BTC_")

In [168]:
pair_data

,open
0,0.62690000
